In [1]:
import pickle
import plotly.plotly as py
import plotly
import plotly.graph_objs as go

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# Family Histogram

In [4]:
with open("pdb_groups.pkl", "r") as f:
    groups = pickle.load(f)

In [61]:
histogram = dict(map(lambda x: (x[0],len(x[1])),groups.items()))

In [62]:
x,y = histogram.keys(), histogram.values()

In [16]:
init_notebook_mode(connected=True)

In [63]:
plotly.offline.plot({
"data": [
    go.Bar(x=x,y=y)
]
})

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp-plot.html'

In [66]:
print "total number of folds: ", len(histogram.keys())
print "total number of pdb files: ", reduce(lambda x,y: x+y, histogram.values())

total number of folds:  4344
total number of pdb files:  16022


# Pdb Vector Length Histogram

In [2]:
from Bio import PDB
import numpy as np
import os
import cv2
#from pdb_utils import *

In [3]:
s_dict = {}
dir_path="PDBs"
files = os.listdir(dir_path)
pdb_files = [(f, os.path.join(dir_path, f)) for f in files if f.endswith(".pdb")]

for pdb, pdb_path in pdb_files:
    parser = PDB.PDBParser()
    structure = parser.get_structure(pdb, pdb_path)  
    s_size = 0
    coords_list = []
    model=structure[0]
    for chain in model.get_list():
        for residue in chain.get_list():
            try:
                coords = residue['CA'].coord
                s_size+=1
            except:
                continue
                
    s_dict[pdb] = s_size

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE1) with given element 'X'

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE2) with given element 'X'



In [4]:
len(s_dict.keys())

16020

In [5]:
hist_dict = {}
for key,value in s_dict.items():
    hist_dict.setdefault(value,0)
    hist_dict[value]+=1

In [6]:
len(hist_dict.keys())

584

In [7]:
x,y = hist_dict.keys(), hist_dict.values()

In [8]:
init_notebook_mode(connected=True)
layout = go.Layout(
    title='PDB Size Histogram',
)

plotly.offline.plot({
"data": [
    go.Bar(x=x,y=y)
]
})

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp-plot.html'

## Fold Histogram

In [5]:
folds = {}
for pclass, pdb_list in groups.items():
    fold = ".".join(pclass.split(".")[:2])
    for pdb in pdb_list:
        folds.setdefault(fold,[])
        folds[fold].append(pdb)

In [6]:
def pdb_size(pdb):
    try:
        dir_path="PDBs"
        pdb_path = os.path.join(dir_path, pdb+".pdb")
        parser = PDB.PDBParser()
        structure = parser.get_structure(pdb, pdb_path)  
        s_size = 0
        model=structure[0]
        for chain in model.get_list():
            for residue in chain.get_list():
                try:
                    coords = residue['CA'].coord
                    s_size+=1
                except:
                    continue
        return s_size
    except:
        return np.nan

In [7]:
fold_hist = dict(map(lambda x:(x[0],map(lambda y:pdb_size(y), x[1])), folds.items()))

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE1) with given element 'X'

/usr/local/lib/python2.7/site-packages/Bio/PDB/Atom.py:185: PDBConstructionWarning:

Could not assign element 'X' for Atom (name=XE2) with given element 'X'



In [9]:
x = fold_hist.keys()
y = map(lambda x: np.mean(x) ,fold_hist.values())

In [10]:
std_dev = map(lambda x: np.nanstd(x) ,fold_hist.values())

In [11]:
mins = map(lambda x: min(x) ,fold_hist.values())
maxs = map(lambda x: max(x) ,fold_hist.values()) 

In [12]:
data = [
    go.Scatter(
        x=x,
        y=y,
        error_y=dict(
            type='data',
            array=std_dev
        )
    )
]

plotly.offline.plot(data, filename='temp.html')

'file:///Users/hacertilbec/Documents/GitHub/Protein-Vectorization/Protein Distance Matrix/temp.html'

In [29]:
trace1 = go.Scatter(
    x=x,
    y=mins,
    name='min ',
    mode = 'markers',
    marker = dict(size = 5)
)
trace2 = go.Scatter(
    x=x,
    y=maxs,
    name='max',
    yaxis='y2',
    mode = 'markers',
    marker = dict(size = 5)
)
trace3 = go.Scatter(
    x=x,
    y=y,
    name="mean",
    yaxis="y3",
    mode = 'markers',
    marker = dict(size = 5),
    error_y=dict(
        type='data',
        thickness=1.5,
        array=std_dev
    )
)
data = [trace1,trace2, trace3]

layout = go.Layout(
    title='Fold based pdb size analysis',
    yaxis2=dict(overlaying='y'),
    yaxis3=dict(overlaying='y')
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='temp2.html')

TypeError: __init__() got an unexpected keyword argument 'shared_yaxes'